In [1]:
# THIS NOTEBOOK NEVER USED, THE IDEA WAS TO ANNOTATE MOTIFS BUT NEVER BEEN FULLY DONE.
# parse motifs annotation file and find its matching gene in rna tsv file

In [2]:
import pandas as pd
import gseapy as gp
from gseapy.parser import Biomart
import matplotlib.pyplot as plt
import os
import gffutils
import glob
import numpy as np
import subprocess as sp
import yaml

In [3]:
print(gp.__version__)

0.10.7


In [4]:
motif_anno = os.path.join("/data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_annotation_MOUSE_mono.tsv")
root = os.path.join("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs")
motif_yaml = os.path.join(root, "tobias_cluster/motif_comparison_clusters.yml")
output_path = os.path.join(root, "annotated_motifs")

In [5]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [6]:
# load the yaml file
with open(motif_yaml, "r") as stream:
    motif_clusters = yaml.safe_load(stream)
motif_clusters.keys()

dict_keys(['Cluster_1', 'Cluster_10', 'Cluster_11', 'Cluster_12', 'Cluster_13', 'Cluster_14', 'Cluster_15', 'Cluster_16', 'Cluster_17', 'Cluster_18', 'Cluster_19', 'Cluster_2', 'Cluster_20', 'Cluster_21', 'Cluster_22', 'Cluster_23', 'Cluster_24', 'Cluster_25', 'Cluster_26', 'Cluster_27', 'Cluster_28', 'Cluster_29', 'Cluster_3', 'Cluster_30', 'Cluster_31', 'Cluster_32', 'Cluster_33', 'Cluster_4', 'Cluster_5', 'Cluster_6', 'Cluster_7', 'Cluster_8', 'Cluster_9'])

In [7]:
count = 0
motif_gene_dict = dict()
anno_df = pd.read_csv(motif_anno, sep = "\t",na_values=np.nan)

for k in motif_clusters.keys():
    motif_gene_dict[k] = dict()
    this_cluster = motif_clusters[k]
    for motif in this_cluster:
        motif = motif.split(" ")[0]
        entrez_id = anno_df.loc[anno_df['Model'] == motif, 'EntrezGene'].str.split(';').values.tolist()[0]
        try:
            l = len(entrez_id)
            entrez_id = list(map(int, entrez_id))
            motif_gene_dict[k][motif] = entrez_id
        except: #nan gene ids
            count += 1
            motif_gene_dict[k][motif] = np.nan
print(count)

10


In [8]:
# convert IDs using biomart
bm = Biomart()
for k, v in motif_gene_dict.items():
    for mo, genes in v.items(): 
        if ~ np.isnan(genes).any():# exclude na 
            queries = [str(i) for i in genes] # need to be a python list
            results = bm.query(dataset='mmusculus_gene_ensembl',
                               attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id'],
                               filters={'entrezgene_id': queries})
            print(results)

      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000048047             Zbtb33          56805
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000043991               Pura          19290
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000038418               Egr1          13653
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000037868               Egr2          13654
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000037868               Egr2          13654
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000033730               Egr3          13655
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000039087              Rreb1          68750
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000050295              Foxc1          17300
      ensembl_gene_id external_gene_name  entrezgene_id
0  ENSMUSG00000078302              Foxd1        